In [227]:
from collections import deque

import gym
from gym import wrappers, logger

import random

import torch

import numpy as np

In [9]:
env_name = 'CartPole-v1'

In [310]:
# def blotzmann(qvalues, tau=0.01):
#     prob = []
#     for qvalue in qvalues:
#         prob.append()
#     print("lol")

def epsilongreedy(qvalues, epsilon=0.5):
    _, max_ind = qvalues.max(0)
    return max_ind.item() if np.random.random() < epsilon else np.random.randint(len(qvalues))

In [301]:
class Agent:
    def __init__(self, observation_size, action_space):
        self.mem = deque(maxlen=10000)
        self.action_space = action_space
        self.observation_size = observation_size
        print(action_space)
        layer_sizes = [
            self.observation_size,
            self.action_space.n
        ]

        self.W = []
        for i in range(len(layer_sizes) - 1):
            self.W.append(torch.ones(layer_sizes[i + 1], layer_sizes[i]).type(torch.FloatTensor))

    def choose_action(self, _observation, _reward, _done):
        qvalues = self.qvalue(_observation)
        action_ind = epsilongreedy(qvalues, epsilon=0.1)
        print(self.action_space.sample(), action_ind)
        return action_ind
        # return self.action_space.sample()

    def experience(self, _state, _action, _reward, _next_state, _done):
        self.mem.append((_state, _action, _reward, _next_state, _done))

    def minibatch(self, size=5):
        batch_s = min(size, len(self.mem))
        return random.sample(self.mem, batch_s)

    def qvalue(self, state):
        inp = torch.tensor(state).type(torch.FloatTensor)
        for w in self.W[:-1]:
            # inp = torch.cat((torch.Tensor([1.0]), inp))
            # inp = func(torch.mv(w, inp))
            inp = 1/(1 + torch.exp(-torch.mv(w, inp)))
        return torch.mv(self.W[-1], inp.float())

In [290]:
env.env.action_space

Discrete(2)

In [311]:
# You can set the level to logger.DEBUG or logger.WARN if you
# want to change the amount of output.
logger.set_level(logger.INFO)

env = gym.make(env_name)

# You provide the directory to write to (can be an existing
# directory, including one with existing data -- all monitor files
# will be namespaced). You can also dump to a tempdir if you'd
# like: tempfile.mkdtemp().
outdir = '/tmp/random-agent-results'
env = wrappers.Monitor(env, directory=outdir, force=True, video_callable=False)
env.seed(0)
observation_size = env.observation_space.shape[0]
action_size = env.action_space.n

agent = Agent(observation_size, env.action_space)

episode_count = 1000
reward = 0
done = False

rewards = []

INFO: Finished writing results. You can upload them to the scoreboard via gym.upload('/tmp/random-agent-results')
INFO: Finished writing results. You can upload them to the scoreboard via gym.upload('/tmp/random-agent-results')
INFO: Making new env: CartPole-v1
INFO: Clearing 4 monitor files from previous run (because force=True was provided)
Discrete(2)


In [312]:
for i in range(episode_count):
    ob = env.reset()
    rewardSum = 0
    steps = 0
    while not done:
        action = agent.choose_action(ob, reward, done)
        current_state = ob
        ob, reward, done, _ = env.step(action)
        rewardSum += reward
        steps += 1

        # Experience replay
        agent.experience(current_state, action, reward, ob, done)
        minibatch = agent.minibatch(size=10)
        if done:
            rewards.append(rewardSum)
            break



env.close()

1
1 0
1
1 1
1
1 1
1
1 0
1
1 1
1
0 1
1
0 1
1
0 0
1
0 0
1
0 1
1
0 0
1
0 1
1
0 1
1
0 1
1
1 1
1
0 1
1
0 1
1
0 0
INFO: Finished writing results. You can upload them to the scoreboard via gym.upload('/tmp/random-agent-results')
